- XGBoost

In [1]:
import pandas as pd
from ydata_profiling import ProfileReport
import lightgbm as lgb
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
import numpy as np
import seaborn as sns
import xgboost as xgb


%matplotlib inline


In [2]:
test_df = pd.read_csv("test.csv")
train_df = pd.read_csv("train.csv")
weather = pd.read_csv("weather.csv")
sample = pd.read_csv("sample_submission.csv")

In [3]:
weather

,date,mean_temp,max_temp,max_temp_time,min_temp,min_temp_time,sum_rain,sun_time,mean_humid,area
0,20041106,13.1,20.1,2004/11/6 12:50,8.1,2004/11/6 05:31,5.0,1.6,77.0,青森
1,20041107,9.6,12.5,2004/11/7 13:40,5.4,2004/11/7 22:17,0.0,4.1,63.0,青森
2,20041108,9.0,15.5,2004/11/8 12:51,3.2,2004/11/8 06:28,0.0,8.9,72.0,青森
3,20041109,12.2,17.7,2004/11/9 14:40,5.7,2004/11/9 02:49,1.5,1.2,85.0,青森
4,20041110,11.6,16.4,2004/11/10 14:42,6.9,2004/11/10 06:23,0.0,7.5,89.0,青森
...,...,...,...,...,...,...,...,...,...,...
204315,20220426,21.3,23.6,2022/4/26 14:27,17.9,2022/4/26 04:17,2.5,0.0,89.0,東京
204316,20220427,23.0,27.7,2022/4/27 13:12,18.9,2022/4/28 0:00,0.5,1.6,79.0,東京
204317,20220428,16.9,21.1,2022/4/28 15:03,14.2,2022/4/28 22:40,0.0,5.5,57.0,東京
204318,20220429,12.9,16.4,2022/4/29 09:21,9.0,2022/4/29 23:55,40.0,0.2,86.0,東京


In [4]:
train_df.groupby

<bound method DataFrame.groupby of        kind      date  amount  mode_price      area
0      だいこん  20051107  201445       735.0        千葉
1      だいこん  20051108  189660       840.0  千葉_各地_青森
2      だいこん  20051110  218166       735.0  千葉_各地_青森
3      だいこん  20051111  182624       682.5     千葉_青森
4      だいこん  20051112  220691       682.5     千葉_青森
...     ...       ...     ...         ...       ...
97777   しめじ  20170526   17231        48.5     新潟_長野
97778   しめじ  20170527   17105        48.5     新潟_長野
97779   しめじ  20170529   12406        48.5     新潟_長野
97780   しめじ  20170530   12320        48.5     新潟_長野
97781   しめじ  20170531   11635        43.0     新潟_長野

[97782 rows x 5 columns]>

In [5]:
sample

,kind,date,mode_price
0,だいこん,20220502,0
1,だいこん,20220506,0
2,だいこん,20220507,0
3,だいこん,20220509,0
4,だいこん,20220510,0
...,...,...,...
315,ミニトマト,20220526,0
316,ミニトマト,20220527,0
317,ミニトマト,20220528,0
318,ミニトマト,20220530,0


In [6]:
#profile = ProfileReport(train_df, title="Profiling Report")
#profile.to_notebook_iframe()
#profile.to_file("your_report.html")
# amount削除
train_df = train_df.drop(columns=['amount'])


In [7]:

#一つ目の産出地のみ出力する
def split_location(series_area):
    x=series_area.str.split("_")
    ret_date = x.apply(lambda x : x[0])
    return ret_date
    

In [8]:


# カテゴリ変数をcategory型に変換 
# 日付データを年数、月、日に分割
def Data_cre(df):

    # areaを一地域のみのデータにする
    df["area"]  = split_location(df["area"])

    # weatherデータと結合
    df= pd.merge(df, weather, on=['date', 'area'], how='left')

    # dateをdatetimeに変更して、年、月、日のデータを取得する
    df["date"] = df["date"].astype("str")
    df["date"] = pd.to_datetime(df["date"])
    
    df["year"] = df["date"].dt.year
    df["month"] = df["date"].dt.month
    df["day"] = df["date"].dt.day
    
    # いらないカラムを削除
    df = df.drop(columns=['date',"max_temp_time","min_temp_time"])
    
    #カテゴリー化
    df['kind'] = df['kind'].astype('category')
    df['area'] = df['area'].astype('category')
    return df

train_df = Data_cre(train_df)
test_df = Data_cre(test_df) 



In [9]:
train_df

,kind,mode_price,area,mean_temp,max_temp,min_temp,sum_rain,sun_time,mean_humid,year,month,day
0,だいこん,735.0,千葉,18.8,24.4,14.7,7.0,5.5,64.0,2005,11,7
1,だいこん,840.0,千葉,17.7,22.0,13.7,0.0,9.5,49.0,2005,11,8
2,だいこん,735.0,千葉,13.6,18.0,9.8,0.0,6.2,64.0,2005,11,10
3,だいこん,682.5,千葉,12.5,14.7,9.5,0.0,0.1,72.0,2005,11,11
4,だいこん,682.5,千葉,13.7,19.3,10.9,10.5,4.9,62.0,2005,11,12
...,...,...,...,...,...,...,...,...,...,...,...,...
97777,しめじ,48.5,新潟,19.8,26.5,15.7,16.5,2.8,77.0,2017,5,26
97778,しめじ,48.5,新潟,17.0,20.7,14.2,4.0,3.2,77.0,2017,5,27
97779,しめじ,48.5,新潟,20.3,27.3,12.3,0.0,12.2,56.0,2017,5,29
97780,しめじ,48.5,新潟,22.4,28.5,17.1,0.0,9.9,60.0,2017,5,30


In [10]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97782 entries, 0 to 97781
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   kind        97782 non-null  category
 1   mode_price  97782 non-null  float64 
 2   area        97782 non-null  category
 3   mean_temp   30282 non-null  float64 
 4   max_temp    30282 non-null  float64 
 5   min_temp    30282 non-null  float64 
 6   sum_rain    30282 non-null  float64 
 7   sun_time    30273 non-null  float64 
 8   mean_humid  30280 non-null  float64 
 9   year        97782 non-null  int32   
 10  month       97782 non-null  int32   
 11  day         97782 non-null  int32   
dtypes: category(2), float64(7), int32(3)
memory usage: 6.5 MB


In [ ]:
### chatgpt
from sklearn.model_selection import KFold
import numpy as np
import lightgbm as lgb

models = {}  # kind → model list or averaged は辞書に保存

K = 5  # ← Fold数

for kd, group in train_df.groupby("kind", observed=False):

    print(f"\n===== {kd} モデルをKFold({K})で学習中 =====")

    X = group.drop(columns=["mode_price"],axis=1)
    y = group["mode_price"]

    kf = KFold(n_splits=K, shuffle=True, random_state=42)

    fold_models = []  # foldごとのモデルを保存

    for fold, (train_index, valid_index) in enumerate(kf.split(X)):
        print(f"Fold {fold+1}/{K}")

        X_train, X_valid = X.iloc[train_index], X.iloc[valid_index]
        y_train, y_valid = y.iloc[train_index], y.iloc[valid_index]

        train_data = lgb.Dataset(X_train, y_train)
        valid_data = lgb.Dataset(X_valid, y_valid)
        
        model = xgb.XGBRegressor(
            n_estimators=1000,
            learning_rate=0.05,
            max_depth=6,
            subsample=0.8,
            colsample_bytree=0.8,
            random_state=42,
            early_stopping_rounds = 10,
            enable_categorical=True
        )

        model.fit(
            X_train, y_train,
            eval_set=[(X_valid, y_valid)],
            verbose=True
        )
        
        fold_models.append(model)

    models[kd] = fold_models  # kindごとにfoldモデルを保存



===== いんげん モデルをKFold(5)で学習中 =====
Fold 1/5
[0]	validation_0-rmse:612.10914
[1]	validation_0-rmse:606.22653
[2]	validation_0-rmse:599.04536
[3]	validation_0-rmse:592.46132
[4]	validation_0-rmse:588.53426
[5]	validation_0-rmse:584.85537
[6]	validation_0-rmse:581.74270
[7]	validation_0-rmse:579.54366
[8]	validation_0-rmse:575.82764
[9]	validation_0-rmse:572.88002
[10]	validation_0-rmse:567.90074
[11]	validation_0-rmse:565.65105
[12]	validation_0-rmse:564.40503
[13]	validation_0-rmse:562.94067
[14]	validation_0-rmse:561.90475
[15]	validation_0-rmse:561.93389
[16]	validation_0-rmse:557.57967
[17]	validation_0-rmse:554.20793
[18]	validation_0-rmse:551.43005
[19]	validation_0-rmse:549.00787
[20]	validation_0-rmse:546.30837
[21]	validation_0-rmse:543.12237
[22]	validation_0-rmse:542.30091
[23]	validation_0-rmse:540.59586
[24]	validation_0-rmse:540.61090
[25]	validation_0-rmse:540.04792
[26]	validation_0-rmse:539.52213
[27]	validation_0-rmse:536.15539
[28]	validation_0-rmse:535.21959
[29]	vali

In [ ]:
import numpy as np

pred_list = []

for kd, group in test_df.groupby("kind"):

    fold_models = models[kd]  # ← foldモデルが複数入っている
    X_test = group.copy()

    # 全 fold モデルの予測を行い平均
    fold_pred = np.mean([m.predict(X_test) for m in fold_models], axis=0)

    X_test["mode_price"] = fold_pred
    pred_list.append(X_test)

# まとめ・並び順合わせ
result = pd.concat(pred_list).sort_index()

# 日付整形（year,month,day → YYYYMMDD）
result["date"] = (
    result["year"].astype(str)
    + result["month"].astype(str).str.zfill(2)
    + result["day"].astype(str).str.zfill(2)
)
result["date"] = result["date"].astype(int)

# 提出形式に整形
submit = result[["kind", "date", "mode_price"]]
submit.to_csv("submit_XGBoost.csv", index=False)

submit.head()